## 实训2　构造信用卡客户风险评价关键特征

1. 训练要点

    1. 掌握信用卡模型的原理。
    2. 构建信用卡用户风险分析关键特征。

2. 需求说明

    &emsp;&emsp;在信用卡相关的征信工作中，主要从3个方向判定客户的信用等级。信用等级分别为：
    
    - 客户的历史信用风险，主要为客户的历史信用情况，包括了用户是否有`逾期`、`呆账`和`强制停卡记录`等
    - 客户的现阶段经济状况，总和考虑了`借款余额`、`个人月收入`、`个人月开销`、`家庭月收入`，以及`月刷卡额`这类和个人经济水平息息相关的特征
    - 客户的未来经济收入以及目前收入的稳定情况，客户的职业不同、年龄不同、房产信息不同，那么客户的经济稳定情况是不同的

3. 实现思路及步骤

    1. 根据特征`瑕疵户`、`逾期`、`呆账`、`强制停卡记录`、`退票`、`拒往记录`，构建历史行为特征。
    2. 根据特征`借款余额`、`个人月收入`、`个人月开销`、`家庭月收入`、`月刷卡额`，构建出经济风险情况特征。
    3. 根据特征`职业`、`年龄`、`住家`，构建出收入风险情况特征。
    4. 标准化历史行为、经济风险情况、收入风险情况特征。

In [1]:
# 导入数据
import pandas
credit_cards = pandas.read_csv('data/credit_card.csv', encoding='GB18030')

In [2]:
# 欧氏距离定义历史行为特征，数值越大信誉越好
import numpy
history_section = credit_cards[['瑕疵户', '逾期', '呆账', '强制停卡记录', '退票', '拒往记录']]
history_section = history_section.to_numpy()
history_section = numpy.array([numpy.linalg.norm(row) for row in history_section])

In [3]:
# 欧氏距离定义经济风险特征，数值越大信誉越好
economy_section = credit_cards[['借款余额', '个人月收入', '个人月开销', '家庭月收入', '月刷卡额']]
economy_section = economy_section.to_numpy()
economy_section = numpy.array([numpy.linalg.norm(row) for row in economy_section])

In [4]:
# 欧式距离定义收入风险情况特征，数值越小信誉越好
income_section = credit_cards[['职业', '年龄', '住家']]
income_section = income_section.to_numpy()
# 特征分量处理
income_section[:, 0] = numpy.abs(10.5 - income_section[:, 0])
income_section[:, 1] = numpy.abs(5 - income_section[:, 1])
income_section[:, 2] = numpy.abs(3.5 - income_section[:, 2])
income_section = numpy.array([numpy.linalg.norm(row) for row in income_section])

In [5]:
eigenvalues = pandas.DataFrame({
    'History activity': history_section,
    'Economy risk': economy_section,
    'Income risk': income_section
})
print('特征表前5行为：', eigenvalues.head(), sep='\n')

特征表前5行为：
   History activity  Economy risk  Income risk
0          4.898979      2.828427     3.000000
1          4.898979      3.741657     8.485281
2          4.898979      6.557439     3.605551
3          4.898979      4.795832     3.000000
4          4.898979      6.324555     2.236068


In [6]:
# 数据标准化
from sklearn.preprocessing import StandardScaler
eigenvalues = StandardScaler().fit_transform(eigenvalues)
print('标准化后的特征表前5行为：', eigenvalues[:5, :], sep='\n')
numpy.savez('stash/credit_card.npz', eigenvalues)

标准化后的特征表前5行为：
[[ 0.31617681 -1.95145645 -0.8854045 ]
 [ 0.31617681 -1.49405487  1.22826534]
 [ 0.31617681 -0.08373941 -0.65206452]
 [ 0.31617681 -0.96605997 -0.8854045 ]
 [ 0.31617681 -0.20038157 -1.1797741 ]]
